## scratch 

In [1]:
#cell 1
import requests  #A
import pandas as pd  
import logging  
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta  
pd.set_option("display.max_colwidth", None)

load_dotenv()

NEWS_API_KEY = os.getenv("NEWS_API_KEY")  #B

# Dynamic date calculation: today minus one day  
today = datetime.now().date()  #C  
yesterday = today - timedelta(days=1)  

# Function to extract articles from NewsAPI  
def extract_articles(query, from_date=yesterday, api_key=NEWS_API_KEY):  
    url = f'https://newsapi.org/v2/everything?q={query}&from={from_date}&to={today}&apiKey={api_key}'  #D
    response = requests.get(url)  
    
    if response.status_code == 200:  
        articles = response.json().get('articles', [])  #E
        logging.info(f"Successfully extracted {len(articles)} articles.")  
        return articles  
    else:  
        logging.error(f"Failed to fetch articles. Status code: {response.status_code}")  
        return []  

# Example use case  
articles = extract_articles('Tesla')  #F

# Build a DataFrame with one row per article and full JSON blob
df = pd.DataFrame({'article': articles})  #G
df



,article
0,"{'source': {'id': None, 'name': 'heise online'}, 'author': 'Frank Schräer', 'title': 'Montag: Neues und günstigeres Tesla Model Y, Kölner KI-Firma vor US-Börsengang', 'description': 'Tesla-Prototyp abgelichtet + Milliardenbewertung für DeepL + VW-Software von Partnern + Fallschirm mit Kirigami-Technik + Ermittler gegen illegales Streaming', 'url': 'https://www.heise.de/news/Montag-Neues-und-guenstigeres-Tesla-Model-Y-Koelner-KI-Firma-vor-US-Boersengang-10712430.html', 'urlToImage': 'https://heise.cloudimg.io/bound/1200x1200/q85.png-lossy-85.webp-lossy-85.foil1/_www-heise-de_/imgs/18/4/9/5/0/4/1/2/monday-0a65d30235cec184.webp', 'publishedAt': '2025-10-06T04:15:00Z', 'content': 'Aufnahmen eines nicht getarnten Tesla auf einem US-Highway und Analysen der neuesten Firmware-Version nähren Spekulationen über ein neues abgespecktes Modell. Auch wenn dieses neue Model Y günstiger … [+7223 chars]'}"
1,"{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Akash Sriram', 'title': 'Tesla teases Tuesday event as focus shifts to affordable EVs', 'description': '(Reuters) -Tesla teased an October 7 event, in which investors and analysts anticipate a more affordable model to sustain sales momentum, sending the shares ...', 'url': 'https://finance.yahoo.com/news/tesla-teases-tuesday-event-focus-043556880.html', 'urlToImage': 'https://media.zenfs.com/en/reuters-finance.com/8158cf25070f1e6e9392d7628f2337f2', 'publishedAt': '2025-10-06T04:35:56Z', 'content': 'By Akash Sriram (Reuters) -Tesla teased an October 7 event, in which investors and analysts anticipate a more affordable model to sustain sales momentum, sending the shares of the Elon Musk-led elec… [+2103 chars]'}"
2,"{'source': {'id': None, 'name': 'Journal du geek'}, 'author': 'Jason Filankembo', 'title': 'Tesla publie une vidéo troublante, une annonce majeure à venir', 'description': 'Et vous, vous comprenez quelque chose à la vidéo de 9 secondes publiée par Tesla ?', 'url': 'https://www.journaldugeek.com/2025/10/06/tesla-publie-video-troublante-annonce-majeure-venir/', 'urlToImage': 'https://www.journaldugeek.com/app/uploads/2025/10/Tesla-logo-1600x900.jpg', 'publishedAt': '2025-10-06T11:06:09Z', 'content': 'Tesla vient de lancer une fonction essentielle en Europe : la détection d’enfant oublié. « Si un enfant est laissé sans surveillance, la fonction de détection denfant oublié déclenche des alarmes de … [+2147 chars]'}"
3,"{'source': {'id': None, 'name': 'Jalopnik'}, 'author': 'staff@jalopnik.com (Andy Kalmowitz)', 'title': 'Tesla Teases New Vehicle Launching This Week That Is Sure To Underwhelm', 'description': 'Tesla seems to be on the precipice of something thoroughly unremarkable.', 'url': 'https://www.jalopnik.com/1987830/tesla-teases-new-vehicle/', 'urlToImage': 'https://www.jalopnik.com/img/gallery/tesla-teases-new-vehicle-launching-this-week-that-is-sure-to-underwhelm/l-intro-1759759493.jpg', 'publishedAt': '2025-10-06T14:05:49Z', 'content': 'There are really only two logical answers for what Tesla is teasing in these videos. It's either a production variant of the long-awaited Roadster or that cheaper, decontented (read: worse) Model Y I… [+1701 chars]'}"
4,"{'source': {'id': None, 'name': 'Golem.de'}, 'author': 'Friedhelm Greis', 'title': 'Elektroautos: Tesla berappelt sich bei Neuzulassungen in Deutschland', 'description': 'Fast jeder fünfte neu zugelassene Pkw im September 2025 ist ein Elektroauto. Plug-in-Hybride legen ebenfalls stark zu. (Elektroauto, Wirtschaft)', 'url': 'https://www.golem.de/sonstiges/zustimmung/auswahl.html?from=https%3A%2F%2Fwww.golem.de%2Fnews%2Felektroautos-tesla-berappelt-sich-bei-neuzulassungen-in-deutschland-2510-200833.html&referer=https%3A%2F%2Ft.co%2F5b9d20d257', 'urlToImage': None, 'publishedAt': '2025-10-06T13:55:02Z', 'content': 'Besuchen Sie Golem.de wie gewohnt mit Werbung und Tracking, indem Sie der Nutzung aller Cookies zustimmen. Details zum Tracking finden Sie im Privacy Center. Skript wurde nicht geladen. 

In [2]:
#cell 2
import os
import logging
import openai
import pandas as pd
from dotenv import load_dotenv
from pydantic import BaseModel

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

class ExtractedArticle(BaseModel):
    source: str
    title: str
    short_summary: str
    publish_date: str

system_prompt = f"""
You are a data extraction agent. For each input article JSON, return a single object matching this schema:
{ExtractedArticle.schema_json(indent=2)}

Use the raw JSON to guide extraction with natural language hints:
- source: use article['source']['name'] when present.
- title: use article['title'].
- short_summary: 1–2 sentences summarizing the article in plain English.
- publish_date: use article['publishedAt'] (ISO-8601 timestamp).

Return exactly one object that matches the schema.
""".strip()

# Sentiment agent (adapted from news_api_tsla_full_pipeline)
def perform_sentiment_analysis(text: str):
    prompt = (
        "Analyze the sentiment of the following text and return a numerical sentiment "
        "score from -1 (very negative) to 1 (very positive). Return only the number: "
        f"{text}"
    )
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            temperature=0.3
        )
        sentiment_str = response.choices[0].message.content.strip()
        return float(sentiment_str)
    except Exception as e:
        logging.error(f"Error performing sentiment analysis: {e}")
        return None

results = []
input_articles = articles  # from prior cell

# Limit for quick iteration; adjust/remove as needed
for idx, article in enumerate(input_articles[:5]):
    try:
        completion = openai.beta.chat.completions.parse(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{article}"}
            ],
            response_format=ExtractedArticle
        )
        parsed = completion.choices[0].message.parsed
        if parsed:
            item = parsed.dict()
            item["sentiment"] = perform_sentiment_analysis(item["short_summary"])  # agent call
            results.append(item)
    except Exception as e:
        print(f"Error on article {idx}: {e}")

extracted_df = pd.DataFrame(results)
extracted_df


/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_8583/3691238272.py:20: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  {ExtractedArticle.schema_json(indent=2)}
/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_8583/3691238272.py:70: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  item = parsed.dict()


,source,title,short_summary,publish_date,sentiment
0,heise online,"Montag: Neues und günstigeres Tesla Model Y, Kölner KI-Firma vor US-Börsengang","The article discusses the appearance of a new, less expensive Tesla Model Y on a US highway, fueling speculation about an upcoming release. Additionally, it highlights DeepL, a Cologne-based AI company, which is preparing for a U.S. IPO.",2025-10-06T04:15:00Z,0.5
1,Yahoo Entertainment,Tesla teases Tuesday event as focus shifts to affordable EVs,"Tesla has announced an upcoming event on October 7, sparking speculation among investors and analysts that the company might unveil a more affordable electric vehicle model. This move is expected to help maintain Tesla's sales momentum.",2025-10-06T04:35:56Z,0.7
2,Journal du geek,"Tesla publie une vidéo troublante, une annonce majeure à venir",Tesla has released a mysterious 9-second video that has left viewers puzzled. The video is expected to precede a major announcement from the company.,2025-10-06T11:06:09Z,0.0
3,Jalopnik,Tesla Teases New Vehicle Launching This Week That Is Sure To Underwhelm,Tesla has hinted at the launch of a new vehicle this week. Speculations suggest it could be either a production version of the anticipated Roadster or a more basic model of the Model Y.,2025-10-06T14:05:49Z,0.2
4,Golem.de,Elektroautos: Tesla berappelt sich bei Neuzulassungen in Deutschland,"In September 2025, nearly one in five newly registered cars in Germany was an electric vehicle, with plug-in hybrids also seeing significant growth. Tesla made notable improvements in new car registrations.",2025-10-06T13:55:02Z,0.8


In [3]:
#cell 3
import logging
import openai
import pandas as pd
from pydantic import BaseModel

class QualityCategorization(BaseModel):
    short_date: str            # YYYY-MM-DD (no timezone)
    publish_est: str           # ISO-8601 datetime in America/New_York
    publish_pst: str           # ISO-8601 datetime in America/Los_Angeles
    publish_gmt: str           # ISO-8601 datetime in GMT/UTC (+00:00)
    topic: str                 # One of: Financial, Operations, Product/Technology, Regulatory/Legal, Market/Competition, Executive/Personnel, Strategy/M&A, Customers/Partnerships, Supply Chain/Manufacturing, ESG/Sustainability, Risk/Incidents, Marketing/PR
    region: str                # One of: North America, South America, Europe, Africa, Middle East, Asia, Oceania

qc_system_prompt = f"""
You are a data quality and categorization agent. For each input article, return a single object matching this schema:
{QualityCategorization.schema_json(indent=2)}

Instructions:
- short_date: Derive from the input publish_date by dropping time and timezone, format as YYYY-MM-DD.
- publish_est / publish_pst / publish_gmt: Convert the input publish_date to the specified timezone and return ISO-8601 (include timezone offset). Use the original timestamp as ground truth. Do not guess.
- topic: Choose the best label from [Financial, Operations, Product/Technology, Regulatory/Legal, Market/Competition, Executive/Personnel, Strategy/M&A, Customers/Partnerships, Supply Chain/Manufacturing, ESG/Sustainability, Risk/Incidents, Marketing/PR]. If none is perfect, pick the closest and be consistent.
- region: Infer using language cues, source, and content (country/city mentions). Map to one of:
  [North America, South America, Europe, Africa, Middle East, Asia, Oceania]. Always use exactly these labels.
- Return strictly valid JSON with exactly these keys and no extra text.
""".strip()

qc_results = []

for idx, row in extracted_df.iterrows():
    article_input = {
        "source": row.get("source", ""),
        "title": row.get("title", ""),
        "short_summary": row.get("short_summary", ""),
        "publish_date": row.get("publish_date", "")
    }
    try:
        completion = openai.beta.chat.completions.parse(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": qc_system_prompt},
                {"role": "user", "content": f"{article_input}"}
            ],
            response_format=QualityCategorization
        )
        parsed = completion.choices[0].message.parsed
        if parsed:
            qc_results.append(parsed.dict())
        else:
            qc_results.append({
                "short_date": "",
                "publish_est": "",
                "publish_pst": "",
                "publish_gmt": "",
                "topic": "",
                "region": ""
            })
    except Exception as e:
        logging.error(f"QC error on row {idx}: {e}")
        qc_results.append({
            "short_date": "",
            "publish_est": "",
            "publish_pst": "",
            "publish_gmt": "",
            "topic": "",
            "region": ""
        })

qc_df = pd.DataFrame(qc_results)
enriched_df = pd.concat([extracted_df.reset_index(drop=True), qc_df], axis=1)
enriched_df


/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_8583/277038693.py:17: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  {QualityCategorization.schema_json(indent=2)}
/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_8583/277038693.py:48: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  qc_results.append(parsed.dict())


,source,title,short_summary,publish_date,sentiment,short_date,publish_est,publish_pst,publish_gmt,topic,region
0,heise online,"Montag: Neues und günstigeres Tesla Model Y, Kölner KI-Firma vor US-Börsengang","The article discusses the appearance of a new, less expensive Tesla Model Y on a US highway, fueling speculation about an upcoming release. Additionally, it highlights DeepL, a Cologne-based AI company, which is preparing for a U.S. IPO.",2025-10-06T04:15:00Z,0.5,2025-10-06,2025-10-05T23:15:00-05:00,2025-10-05T21:15:00-07:00,2025-10-06T04:15:00+00:00,Product/Technology,Europe
1,Yahoo Entertainment,Tesla teases Tuesday event as focus shifts to affordable EVs,"Tesla has announced an upcoming event on October 7, sparking speculation among investors and analysts that the company might unveil a more affordable electric vehicle model. This move is expected to help maintain Tesla's sales momentum.",2025-10-06T04:35:56Z,0.7,2025-10-06,2025-10-05T23:35:56-05:00,2025-10-05T20:35:56-08:00,2025-10-06T04:35:56Z,Product/Technology,North America
2,Journal du geek,"Tesla publie une vidéo troublante, une annonce majeure à venir",Tesla has released a mysterious 9-second video that has left viewers puzzled. The video is expected to precede a major announcement from the company.,2025-10-06T11:06:09Z,0.0,2025-10-06,2025-10-06T07:06:09-04:00,2025-10-06T04:06:09-07:00,2025-10-06T11:06:09Z,Marketing/PR,North America
3,Jalopnik,Tesla Teases New Vehicle Launching This Week That Is Sure To Underwhelm,Tesla has hinted at the launch of a new vehicle this week. Speculations suggest it could be either a production version of the anticipated Roadster or a more basic model of the Model Y.,2025-10-06T14:05:49Z,0.2,2025-10-06,2025-10-06T10:05:49-04:00,2025-10-06T07:05:49-07:00,2025-10-06T14:05:49Z,Product/Technology,North America
4,Golem.de,Elektroautos: Tesla berappelt sich bei Neuzulassungen in Deutschland,"In September 2025, nearly one in five newly registered cars in Germany was an electric vehicle, with plug-in hybrids also seeing significant growth. Tesla made notable improvements in new car registrations.",2025-10-06T13:55:02Z,0.8,2025-10-06,2025-10-06T09:55:02-04:00,2025-10-06T06:55:02-07:00,2025-10-06T13:55:02Z,Market/Competition,Europe


In [7]:
#cell 4
import os
import json
import logging
import pandas as pd
import openai
from pydantic import BaseModel

# Ensure psycopg is available
import sys, subprocess
try:
    import psycopg
except Exception:
    subprocess.run([sys.executable, "-m", "pip", "install", "psycopg[binary]>=3.1"], check=False)
    import psycopg

openai.api_key = os.getenv("OPENAI_API_KEY")

# Pydantic for DDL contract
class TableDDL(BaseModel):
    ddl: str  # CREATE TABLE ... statement only

# Model-friendly schema of enriched_df
sample_fields = {
    "source": "text",
    "title": "text",
    "short_summary": "text",
    "publish_date": "timestamptz",
    "sentiment": "numeric",
    "short_date": "date",
    "publish_est": "timestamptz",
    "publish_pst": "timestamptz",
    "publish_gmt": "timestamptz",
    "topic": "text",
    "region": "text"
}

# Compose prompt to generate DDL
ddl_prompt = f"""
You are a SQL DDL assistant. Return only a single valid PostgreSQL CREATE TABLE statement for table name news_articles.
Use these columns and suggested types. Adjust types conservatively if needed, add NOT NULL only if obviously safe.
Columns:
{json.dumps(sample_fields, indent=2)}

Rules:
- Include a surrogate primary key id BIGSERIAL PRIMARY KEY.
- Add created_at TIMESTAMPTZ DEFAULT NOW().
- Use snake_case column names exactly as provided.
- Return strictly the SQL, no comments or extra text.
""".strip()

# Ask AI for DDL
completion = openai.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": ddl_prompt},
        {"role": "user", "content": "Generate the DDL now."}
    ],
    response_format=TableDDL
)
TABLE_DDL = completion.choices[0].message.parsed.ddl
print(TABLE_DDL)

# Connect to Postgres
conn = psycopg.connect(
    host=os.getenv("PGHOST", "localhost"),
    port=os.getenv("PGPORT", "5432"),
    dbname=os.getenv("PGDATABASE", "news_db"),
    user=os.getenv("PGUSER", "news_user"),
    password=os.getenv("PGPASSWORD", "")
)

# Create table if not exists (idempotent):
with conn.cursor() as cur:
    try:
        cur.execute(TABLE_DDL)
    except Exception as e:
        # If table already exists, ignore
        msg = str(e).lower()
        if "already exists" not in msg:
            raise
conn.commit()

# Prepare insert (upsert optional)
cols = [
    "source", "title", "short_summary", "publish_date", "sentiment",
    "short_date", "publish_est", "publish_pst", "publish_gmt", "topic", "region"
]

placeholders = ",".join(["%s"] * len(cols))
insert_sql = f"INSERT INTO news_articles ({','.join(cols)}) VALUES ({placeholders})"

# Convert dataframe rows to tuples
rows = []
for _, r in enriched_df.iterrows():
    rows.append(tuple(r.get(c) for c in cols))

# Batch insert
with conn.cursor() as cur:
    if rows:
        cur.executemany(insert_sql, rows)
        print(f"Inserted {len(rows)} rows into news_articles")
    else:
        print("No rows to insert")
conn.commit()

conn.close()



CREATE TABLE news_articles (
    id BIGSERIAL PRIMARY KEY,
    source TEXT,
    title TEXT,
    short_summary TEXT,
    publish_date TIMESTAMPTZ,
    sentiment NUMERIC,
    short_date DATE,
    publish_est TIMESTAMPTZ,
    publish_pst TIMESTAMPTZ,
    publish_gmt TIMESTAMPTZ,
    topic TEXT,
    region TEXT,
    created_at TIMESTAMPTZ DEFAULT NOW()
);
Inserted 5 rows into news_articles


In [9]:
#cell 5
import os
import pandas as pd

# Reuse psycopg from previous cell (installed if needed there)
import psycopg

conn = psycopg.connect(
    host=os.getenv("PGHOST", "localhost"),
    port=os.getenv("PGPORT", "5432"),
    dbname=os.getenv("PGDATABASE", "news_db"),
    user=os.getenv("PGUSER", "news_user"),
    password=os.getenv("PGPASSWORD", "")
)

# Show total rows
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM news_articles;")
    total_rows = cur.fetchone()[0]
print(f"news_articles rows: {total_rows}")

# Preview last 5 rows
with conn.cursor() as cur:
    cur.execute(
        """
        SELECT id, source, title, short_summary, publish_date, topic, region, sentiment, created_at
        FROM news_articles
        ORDER BY id DESC
        LIMIT 5;
        """
    )
    rows = cur.fetchall()
    cols = [c[0] for c in cur.description]

conn.close()

pd.DataFrame(rows, columns=cols)


news_articles rows: 5


,id,source,title,short_summary,publish_date,topic,region,sentiment,created_at
0,5,Golem.de,Elektroautos: Tesla berappelt sich bei Neuzulassungen in Deutschland,"In September 2025, nearly one in five newly registered cars in Germany was an electric vehicle, with plug-in hybrids also seeing significant growth. Tesla made notable improvements in new car registrations.",2025-10-06 09:55:02-04:00,Market/Competition,Europe,0.8,2025-10-07 17:52:54.607671-04:00
1,4,Jalopnik,Tesla Teases New Vehicle Launching This Week That Is Sure To Underwhelm,Tesla has hinted at the launch of a new vehicle this week. Speculations suggest it could be either a production version of the anticipated Roadster or a more basic model of the Model Y.,2025-10-06 10:05:49-04:00,Product/Technology,North America,0.2,2025-10-07 17:52:54.607671-04:00
2,3,Journal du geek,"Tesla publie une vidéo troublante, une annonce majeure à venir",Tesla has released a mysterious 9-second video that has left viewers puzzled. The video is expected to precede a major announcement from the company.,2025-10-06 07:06:09-04:00,Marketing/PR,North America,0,2025-10-07 17:52:54.607671-04:00
3,2,Yahoo Entertainment,Tesla teases Tuesday event as focus shifts to affordable EVs,"Tesla has announced an upcoming event on October 7, sparking speculation among investors and analysts that the company might unveil a more affordable electric vehicle model. This move is expected to help maintain Tesla's sales momentum.",2025-10-06 00:35:56-04:00,Product/Technology,North America,0.7,2025-10-07 17:52:54.607671-04:00
4,1,heise online,"Montag: Neues und günstigeres Tesla Model Y, Kölner KI-Firma vor US-Börsengang","The article discusses the appearance of a new, less expensive Tesla Model Y on a US highway, fueling speculation about an upcoming release. Additionally, it highlights DeepL, a Cologne-based AI company, which is preparing for a U.S. IPO.",2025-10-06 00:15:00-04:00,Product/Technology,Europe,0.5,2025-10-07 17:52:54.607671-04:00
